# <center>Data Science Project Part 1:  Differential Privacy</center>
<center>DATA 558, Spring 2021</center>
<center>TAs: Alec Greaves-Tunnell and Ronak Mehta</center>


#### Name:  Emily Yamauchi
#### Partner:  Samuel Perebikovsky
#### Summary of findings:

    A: Preprocessing: Should exclude testing set from `database`
                      Should binary variables get standardized?
                      Should X_test be excluded from standardization scale?

    B: Clearly there should be difference between the three methods, as Mech1 only passes the variables, whereas Mech2 and Mech3 adds noise.
    Posterior calculation: numerator should be P(label|flipper)*P(flipper)
    Conditional probability setup: See below.
    
    C: The three methods differ when the above two steps are implemented. Mech2 is approx. 5% vs 12% prior probability, whereas Mech1 and Mech3 are 0% (no privacy maintained?)
    When conditional probability setup change is implemented, prior is 87% (i.e. P(no flipper)), posterior is 100%, 97%, 100% respectively.
    
    ...

In [1]:
import numpy as np
import pickle
import matplotlib.pyplot as plt

## Dataset: Animals with Attributes v2

The data comes from Animals with Attributes v2 dataset, which contains images of 50 different types of animals, with each animal labeled with various attributes (whether it flies, whether it has a tail, etc.). In this example, we will not use the images, and treat the classes themselves as datapoints. That is, we will have $n = 50$ data points $(x_1, y_1), ..., (x_n, y_n)$, where $x_i \in \{0, 1\}^d$ is a binary vector of attributes, and $y_i \in \{0, 1\}$ is a binary label indicating whether the animal is an `ocean` animal or not. There were originally 85 attributes, but we have subsetted them to $d = 5$ features, namely:

- `horns` - whether the animal has horns.
- `tree` - whether the animal lives in a tree.
- `bulbous` - whether the animal is stocky.
- `fierce` - whether the animal is fierce.
- `arctic` - whether the animal lives in the arctic.

Additionally, we have one protected attribute, `flippers`, indicating whether the animal has flippers. This attribute is known for 49 of the animals, but is unknown for a held out animal, the `buffalo`. We will inspect in this notebook whether we can uncover the `buffalo`'s protected attribute using a machine learning model trained on the 49 points. If the model is privacy preserving, we should not be able to do this any better than if we did not have the model in hand. If not, then we will be significantly more confident by virtue of having access to the (outputs of the) model.

In [2]:
data = pickle.load(open("privacy_data.pkl", "rb"))

X = data['X'].to_numpy() 
y = data['y'].to_numpy() # ocean
attr = data['z'].to_numpy()   # The attributes of the database entries. 
x = data['animal'].to_numpy() # The targeted individual.

In [3]:
df = data['X'].reset_index().sort_values(by = 1).rename(columns = {1:'animal'})

The analysis you are expected to critique begins below.

## <center>Differentially Private Logistic Regression for AwA2</center>

## Preprocessing

Standard preprocessing techniques are applied.

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

np.random.seed(123)

X_train, X_test, y_train, y_test, attr_train, attr_test = train_test_split(X, y, attr, test_size=0.2)
scaler = StandardScaler().fit(X)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print("X train shape:", X_train.shape)
print("y train shape:", y_train.shape)
print("X test shape:", X_test.shape)
print("y test shape:", y_test.shape)
print("attr train shape: ", attr_train.shape)
print("attr test shape: ", attr_test.shape)

X train shape: (39, 5)
y train shape: (39,)
X test shape: (10, 5)
y test shape: (10,)
attr train shape:  (39,)
attr test shape:  (10,)


##### Comment 1:  
- `StandardScaler().fit()` should be called on `X_train` to avoid scaling on entire dataset (i.e. peeking at the test set).
  However-- Lab4 also called the `StandardScaler().fit()` to the entire `X` set, so not confident on this critique.
- Do binary inputs need to be scaled?
- `z` also needs to be split?

In [5]:
#?train_test_split

## The Privacy Mechanism

Below, we will implement the mechanism that returns responses from the machine learning model given queries from the data analyst.

In [6]:
from abc import ABC, abstractmethod
from sklearn.linear_model import LogisticRegression

class Mechanism:
    
    @abstractmethod
    def __init__(self, database, **kwargs):
        pass
    
    @abstractmethod
    def respond(query):
        pass

In [7]:
#?ABC

First, we cover the machine learning model. We will use unregularied logistic regression to map the feature vector $x \in \{0, 1\}^5$ to its label $y \in \{0, 1\}$. The `query` from the data analyst can come in four forms.
- `all` - indicating that the mechanism should return responses (predicted labels) for all 49 training points in the database.
- `flippers` - indicating that the mechanism should return responses (predicted labels) for all training points in the database that have flippers.
- `no_flippers` - indicating that the mechanism should return responses (predicted labels) for all training points in the database that do not have flippers.
- `x` - a single feature vector to be predicted by the model, passed as a `numpy.ndarray`.

We implement three mechanisms, each of which will produce the response in different ways while preserving privacy.

In [8]:
class Mechanism1(Mechanism):
    
    def __init__(self, database, attr):
        self.X, self.y = database
        self.model = LogisticRegression().fit(self.X, self.y)
        self.attr = attr
        self.name = "Mechanism 1"
        
    def respond(self, query):
        if isinstance(query, np.ndarray):
            return self.model.predict(query.reshape(1, -1))[0]
        elif query == "all":
            return self.model.predict(self.X)
        elif query == "flippers":
            return self.model.predict(self.X[self.attr == 1])
        elif query == "no_flippers":
            return self.model.predict(self.X[self.attr == 0])
        else:
            raise ValueError("'query' must be 'all', 'flippers', 'no_flippers', or a numpy.ndarry object.")

##### Comment 2:  
`Mechanism1` has no privacy protection- passes each variable through the class with no noise added.

In [9]:
class Mechanism2(Mechanism):
    
    def __init__(self, database, attr, prob=0.9):
        self.X, self.y = database
        self.model = LogisticRegression().fit(self.X, self.y)
        self.attr = attr
        self.name = "Mechanism 2"
        self.prob = prob
        
    def respond(self, query):
        if isinstance(query, np.ndarray):
            coin = np.random.binomial(1, self.prob)
            if coin == 1:
                return self.model.predict(query.reshape(1, -1))[0]
            else:
                return np.random.binomial(1, 0.5)
        elif query == "all":
            X = self.X
        elif query == "flippers":
            X = self.X[self.attr == 1]
        elif query == "no_flippers":
            X = self.X[self.attr == 0]
        else:
            raise ValueError("'query' must be 'all', 'flippers', 'no_flippers', or a numpy.ndarry object.")
            
        y = self.model.predict(X)
        coins = np.random.binomial(1, self.prob, size=y.shape)
        random_response = np.random.binomial(1, 0.5, size=y.shape)
        
        return y * coins + random_response * (1 - coins)

##### Comment 3:   
`Mechanism2` adds noise to the output directly. Adds a weighted coin-toss at default 90% to the output. Possible critique-- is this enough nosie? TBD?

In [10]:
        
class Mechanism3(Mechanism):
    
    def __init__(self, database, attr, scale=1):
        self.X, self.y = database
        self.model = LogisticRegression().fit(self.X, self.y)
        self.attr = attr
        self.name = "Mechanism 3"
        self.scale = scale
        
    def respond(self, query):
        if isinstance(query, np.ndarray):
            noise = np.random.uniform(-0.1, 0.1, size=query.shape)
            return  self.model.predict((query + noise).reshape(1, -1))[0]
        elif query == "all":
            X = self.X
        elif query == "flippers":
            X = self.X[self.attr == 1]
        elif query == "no_flippers":
            X = self.X[self.attr == 0]
        else:
            raise ValueError("'query' must be 'all', 'flippers', 'no_flippers', or a numpy.ndarry object.")
            
        X = X + np.random.uniform(-0.1, 0.1, size=X.shape)
        return self.model.predict(X)

##### Comment 4:  
`Mechanism3` adds noise to the input variable as uniform distribution. 

In [11]:
database = (X, y)

m1 = Mechanism1(database, attr)
m2 = Mechanism2(database, attr)
m3 = Mechanism3(database, attr)

In [12]:
data_fix = (X_train, y_train)
m1_fix = Mechanism1(data_fix, attr_train)
m2_fix = Mechanism2(data_fix, attr_train)
m3_fix = Mechanism3(data_fix, attr_train)

##### Comment 5:  
`Database` should consist of just `X_train`, `y_train`.

In [13]:
print('m1.respond(x): ', m1.respond(x))
print('m1_fix.respond(x): ', m1_fix.respond(x))

m1.respond(x):  0
m1_fix.respond(x):  0


In [14]:
print('m2.respond(x): ', m2.respond(x))
print('m2_fix.respond(x): ', m2_fix.respond(x))

m2.respond(x):  0
m2_fix.respond(x):  0


In [15]:
print('m3.respond(x): ', m3.respond(x))
print('m3_fix.respond(x): ', m3_fix.respond(x))

m3.respond(x):  0
m3_fix.respond(x):  0


## The Advarsarial Attack

We justify our claim of privacy by showing that an attack fails. In other words, we want the probabilities surrounding whether the targetted animal (`buffalo`) having the attribute `flipper` to not change very much given the response $\hat{y}$ from the mechanism. The prior probability (that is, prior to calling the mechanism) of having `flippers` is estimated using the number of animals in the training set that have flippers, which we assume is known by the attacker. We want:

$$
\mathbb{P}\left(\text{flippers}\mid \hat{y}(\text{buffalo}) =\text{ocean}\right) \approx \mathbb{P}\left(\text{flippers}\right)
$$

If we are much more confident about the value of this protected attribute (i.e. the probabilities are significantly higher or lower) after using the prediction for the `ocean` attribute of `buffalo` from the model, then the mechanism has failed to protect privacy.

In [16]:
# Use Bayes rule to get a value for the probability of `x` having attr == flippers.
np.random.seed(123)

prior = attr.mean()
print("Prior: %0.3f" % prior)

for mech in [m1, m2, m3]:
    label = mech.respond(x)
    # print(label)

    prob_1_given_attr_1 = mech.respond("flippers").mean()
    # print(prob_1_given_attr_1)
    prob_1_given_attr_0 = mech.respond("no_flippers").mean()
    # print(prob_1_given_attr_0)

    if label == 1:
        prob_label_given_attr_1 = prob_1_given_attr_1
        prob_label_given_attr_0 = prob_1_given_attr_0
    else:
        prob_label_given_attr_1 = prob_1_given_attr_0
        prob_label_given_attr_0 = prob_1_given_attr_1

    posterior = prior * prob_label_given_attr_0 / (prior * prob_label_given_attr_1 + (1 - prior) *  prob_label_given_attr_0)

    print("%s Posterior: %0.3f" % (mech.name, posterior))

Prior: 0.143
Mechanism 1 Posterior: 0.167
Mechanism 2 Posterior: 0.167
Mechanism 3 Posterior: 0.166


In [17]:
# Use Bayes rule to get a value for the probability of `x` having attr == flippers.
#np.random.seed(123)
# Use simulation instead. Fix posterior calc only. Use trainset

prior = attr_train.mean()
print("Prior: %0.3f" % prior)

for mech in [m1_fix, m2_fix, m3_fix]:
    post = []
    
    for i in range(1000):
        label = mech.respond(x)
        # print(label)

        prob_1_given_attr_1 = mech.respond("flippers").mean()
        # print(prob_1_given_attr_1)
        prob_1_given_attr_0 = mech.respond("no_flippers").mean()
        # print(prob_1_given_attr_0)

        if label == 1:
            prob_label_given_attr_1 = prob_1_given_attr_1
            prob_label_given_attr_0 = prob_1_given_attr_0
        else:
            prob_label_given_attr_1 = prob_1_given_attr_0
            prob_label_given_attr_0 = prob_1_given_attr_1

        post_calc = prior * prob_label_given_attr_1 / (prior * prob_label_given_attr_1 + (1 - prior) *  prob_label_given_attr_0)
        post.append(post_calc)
    
    posterior = np.mean(np.array(post))
    print("%s Posterior: %0.3f" % (mech.name, posterior))

Prior: 0.128
Mechanism 1 Posterior: 0.004
Mechanism 2 Posterior: 0.047
Mechanism 3 Posterior: 0.005


$$
\begin{align}
P(\text{flippers}|\text{label})&=\frac{P(\text{label}|\text{flippers})\times P(\text{flippers})}{P(\text{label}|\text{flippers})\times P(\text{flippers})+P(\text{label}|\text{no flippers})\times P(\text{no flippers})}
\end{align}
$$
$$
\begin{align}
\\ \\ \\ 
\small\text{Case label == 1:}\\ 
P(\text{flippers}|\text{ocean})&=\frac{P(\text{ocean}|\text{flippers})\times P(\text{flippers})}{P(\text{ocean}|\text{flippers})\times P(\text{flippers})+P(\text{ocean}|\text{no flippers})\times P(\text{no flippers})}
\\ \\ \\ 
\small\text{Case label == 0:}\\ 
P(\text{no flippers}|\text{no ocean})&=\frac{P(\text{no ocean}|\text{no flippers})\times P(\text{no flippers})}{P(\text{no ocean}|\text{no flippers})\times P(\text{no flippers})+P(\text{no ocean}|\text{flippers})\times P(\text{flippers})}
\end{align}
$$

In [18]:
# Fix calc?
#np.random.seed(123)

flippers = attr_train.mean() #P(flippers)



for mech in [m1_fix, m2_fix, m3_fix]:
    post = []
    
    for i in range(1000):
        label = mech.respond(x) #predict if buffalo is ocean based on mechanism 1-3
        #print("label (buffalo = ocean): ", label)

        given_flippers = mech.respond("flippers").mean() #P(ocean|flippers)
        given_not_flippers = mech.respond("no_flippers").mean() #P(ocean|no flippers)

        if label == 1: # label = ocean (case 1)
            label_given_f = given_flippers
            not_label_given_f = given_not_flippers
            prior = flippers

        else: # label = not ocean (case 0)
            label_given_f = 1- given_not_flippers
            not_label_given_f = 1- given_flippers
            prior = 1 - flippers

        post_calc = prior*label_given_f / (prior*label_given_f + (1-prior)*not_label_given_f)
        post.append(post_calc)
        #posterior = post_calc
    
    posterior = np.array(post).mean()
    print("Prior: %0.3f" % prior)
    print("%s Posterior: %0.3f" % (mech.name, posterior))
    print("\n")

Prior: 0.872
Mechanism 1 Posterior: 1.000


Prior: 0.872
Mechanism 2 Posterior: 0.977


Prior: 0.872
Mechanism 3 Posterior: 1.000




Clearly, these probabilities have not changed very much from the prior. Thus, we can be assured that each mechanism preserves privacy.

The three mechanisms should have different results, or at least, Mech1 has no privacy masking mechanism. As shown above, Mech2 has some privacy masking mechanism, while Mech1 and Mech3 do not. Therefore, masking the input only is not sufficient to maintain privacy, but the output variable themselves must have noise injected.

## Maintaining Accuracy

Trivially, we can always preserve privacy by injecting a sufficiently large amount of noise. While this may be good by one metric, we might completely destroy the predictive performance of the model! It is important to maintain a balance such that we still perform well on a test set, which we inspect below. Note that this step would normally be done on a validation set, and final performance would be computed on the test set.

In [19]:
#?accuracy_score

In [20]:
from sklearn.metrics import accuracy_score

np.random.seed(123)

for mech in [m1, m2, m3]:
    y_pred = np.array([mech.respond(x) for x in X_test])
    print("%s Test Accuracy: %0.2f" % (mech.name, accuracy_score(y_pred, y_test)))

Mechanism 1 Test Accuracy: 0.90
Mechanism 2 Test Accuracy: 0.80
Mechanism 3 Test Accuracy: 0.90
